In [1]:
setprecision(256)

C = 0.47

0.47

## Generator testów

In [2]:
function gen_test(satellites)
    # wektor [x; y; z; t] - (x, y, z): nasza pozycja, t: błąd zegara
    X = vcat(normalize(Array{BigFloat}(2 * rand(3) - 1) .^ 3), Array{BigFloat}(randn(1)) .^ 3)
    data = Array{BigFloat, 2}(4, satellites)
    for i = 1:satellites
        v = normalize(Array{BigFloat}(2 * rand(3) - 1) .^ 3)
        data[:, i] = X + vcat(v, norm(v) / C)
    end
    return data, X
end

gen_test (generic function with 1 method)

## Dane do testów

In [3]:
n = 8 # liczba satelit

data, solution₁ = gen_test(n)

# data[1, :] : wektor współrzędnych x satelitów
# data[2, :] : wektor współrzędnych y satelitów
# data[3, :] : wektor współrzędnych z satelitów
# data[4, :] : wektor czasów zegara satelitów

# data[:, i] : dane z i-tej satelity
# solution₁ = poszukiwane rozwiązanie (niekoniecznie jedyne)

# wyświetlanie wyników
function prnt(x::Array{BigFloat}, fx::Array{BigFloat})
    @printf "x - sol: [%.4e, %.4e, %.4e, %.4e]\tf(x): [%.4e, %.4e, %.4e, %.4e]\n" x[1] x[2] x[3] x[4] fx[1] fx[2] fx[3] fx[4]
end

# Funkcja F jak w sprawozdaniu
# rozważamy dane tylko z 4 pierwszych satelitów
function f4(x::Array{BigFloat})
    return ((-data[:, 1:4] .+ x) .^ 2)' * [1, 1, 1, -C*C]
end


# Funkcja F jak w sprawozdaniu
# bierze pod uwagę dane ze wszystkich satelitów
function f(x::Array{BigFloat})
    return ((-data .+ x) .^ 2)' * [1, 1, 1, -C*C]
end

# Jakobian jak w sprawozdaniu
# dla 4 pierwszych satelitów
∂f4arr = Array{Function, 2}(4, 4)
for i = 1:4
    for j = 1:3
        ∂f4arr[i, j] = function(x) return 2*x[j] - 2*data[j, i] end
    end
    ∂f4arr[i, 4] = function(x) return 2*C*C*data[4, i] - 2*C*C*x[4] end
end

function ∂f4(x::Array{BigFloat})
    return map((f) -> f(x), ∂f4arr)
end


# Jakobian jak w sprawozdaniu
# dla wszystkich satelitów
∂farr = Array{Function, 2}(n, 4)
for i = 1:n
    for j = 1:3
        ∂farr[i, j] = function(x) return 2*x[j] - 2*data[j, i] end
    end
    ∂farr[i, 4] = function(x) return 2*C*C*data[4, i] - 2*C*C*x[4] end
end

function ∂f(x::Array{BigFloat})
    return map((f) -> f(x), ∂farr)
end

∂f (generic function with 1 method)

# Metoda Newtona

In [4]:
function NewtonMethod(f, ∂f, ϵ; x=Array{BigFloat}([0, 0, 0, 0]), max_iter=1000, log=true, solution=solution₁)
    i = 0
    while norm(f(x)) > ϵ && i < max_iter
        δ = ∂f(x) \ f(x)
        x = x - δ
        i += 1
        if log prnt(abs.(x - solution), abs.(f(x))) end
    end
    return x
end

newton_4_result = NewtonMethod(f4, ∂f4, 1e-70)
f(newton_4_result)

x - sol: [8.3663e-78, 3.0227e-77, 3.4545e-77, 1.8602e+00]	f(x): [9.8420e-01, 9.8420e-01, 9.8420e-01, 9.8420e-01]
x - sol: [1.2684e-77, 8.6362e-78, 8.6362e-78, 6.4692e+00]	f(x): [1.5326e+01, 1.5326e+01, 1.5326e+01, 1.5326e+01]
x - sol: [1.0512e-76, 1.0363e-76, 8.6362e-78, 2.4341e+00]	f(x): [3.5968e+00, 3.5968e+00, 3.5968e+00, 3.5968e+00]
x - sol: [1.4978e-77, 5.1817e-77, 2.5909e-77, 6.4939e-01]	f(x): [7.0358e-01, 7.0358e-01, 7.0358e-01, 7.0358e-01]
x - sol: [1.8892e-78, 3.8863e-77, 8.6362e-78, 7.5926e-02]	f(x): [7.2644e-02, 7.2644e-02, 7.2644e-02, 7.2644e-02]
x - sol: [6.2072e-78, 4.3181e-77, 1.7272e-77, 1.3081e-03]	f(x): [1.2300e-03, 1.2300e-03, 1.2300e-03, 1.2300e-03]
x - sol: [4.0482e-79, 4.3181e-77, 8.6362e-78, 4.0184e-07]	f(x): [3.7773e-07, 3.7773e-07, 3.7773e-07, 3.7773e-07]
x - sol: [3.5084e-78, 3.8863e-77, 8.6362e-78, 3.7897e-14]	f(x): [3.5670e-14, 3.5670e-14, 3.5670e-14, 3.5670e-14]
x - sol: [7.4217e-78, 3.4545e-77, 8.6362e-78, 4.9617e-17]	f(x): [3.1809e-28, 3.1809e-28, 3.1809e

8-element Array{BigFloat,1}:
 1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77
 1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77
 0.000000000000000000000000000000000000000000000000000000000000000000000000000000    
 1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77
 3.454467422037777850154540745120159828446400145774512554009481388067436721264971e-77
 8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-77
 8.636168555094444625386351862800399571116000364436281385023703470168591803162427e-78
 3.454467422037777850154540745120159828446400145774512554009481388067436721264971e-77

### Test metody Newtona

In [5]:
solved_count = 0
test_count = 100000 # zmienić na true, żeby uruchomić testy
TEST = false

error = Array{BigFloat}(test_count)
max_error = BigFloat(0)
min_error = BigFloat(Inf)

if TEST for p = 1:test_count
    if p%(test_count/100)==0
        @printf "%d " (100*p/test_count)
    end
    temp_data, temp_solution₁ = gen_test(4)
    
    function temp_f(x::Array{BigFloat})
        return ((-temp_data .+ x) .^ 2)' * [1, 1, 1, -C*C]
    end
    
    temp_∂f4arr = Array{Function, 2}(4, 4)
    for i = 1:4
        for j = 1:3
            temp_∂f4arr[i, j] = function(x) return 2*x[j] - 2*temp_data[j, i] end
        end
        temp_∂f4arr[i, 4] = function(x) return 2*C*C*temp_data[4, i] - 2*C*C*x[4] end
    end
    
    function temp_∂f(x::Array{BigFloat})
        return map((f) -> f(x), temp_∂f4arr)
    end
    
    temp_result = NewtonMethod(temp_f, temp_∂f, 1e-70; log=false)
        
    error[p] = norm(temp_solution₁[1:3] - temp_result[1:3])
    if error[p] > max_error max_error = error[p] end
    if error[p] < min_error min_error = error[p] end
    if(error[p] < 1e-70) solved_count += 1 end
end
    @printf "\nsolved: %d / %d" solved_count test_count
    @printf "\nerror: max: %.4e avg: %.4e min: %.4e" max_error mean(error) min_error
end


solved: 0 / 100000

LoadError: [91mUndefRefError: access to undefined reference[39m

## Metoda najmniejszych kwadratów na pomocą metody Newtona dla wielu satelit

In [11]:
function S(x::Array{BigFloat})
    return [sum(4 * f(x) .* (x[i] - data[i, :])) for i = 1:4] .* [1, 1, 1, -C*C]
end

∂Sarr = Array{Function, 2}(4, 4)
for i = 1:4
    for j = 1:4
        ∂Sarr[i, j] = (i == 4 || j == 4) ? 
            function(x) return - C * C * 8 * sum((x[j] - data[j, :]) .* (x[i] - data[i, :])) end :
            function(x) return 8 * sum((x[j] - data[j, :]) .* (x[i] - data[i, :])) end

    end
end

for i = 1:3
    ∂Sarr[i, i] = function(x) return 4 * sum(f(x) + (2 * (data[i, :] - x[i]) .* (data[i, :] - x[i]))) end
end

∂Sarr[4, 4] = function(x) return - 4 * C * C * sum(f(x) - (2 * C * C * (data[4, :] - x[4]) .* (data[4, :] - x[4]))) end

# Jakobian jak w sprawozdaniu
function ∂S(x::Array{BigFloat})
    return map((f) -> f(x), ∂Sarr)
end

newton_least²_result = NewtonMethod(S, ∂S, 1e-70; log=false)
@show norm(S(newton_least²_result))
@show norm(f(newton_least²_result))
@show norm(f(newton_4_result))

norm(S(newton_least²_result)) = 4.251649535958333306506034500366618328302946483196454361241690618740310568684449e-76
norm(f(newton_least²_result)) = 2.720916450683171049841241776236413290322069713657881253875382833283803963536402
norm(f(newton_4_result)) = 1.039932404545617559908005063501300374894691794516581500560097001859466064289471e-76


1.039932404545617559908005063501300374894691794516581500560097001859466064289471e-76

### Test metody najmniejszych kwadratów

In [7]:
solved_count = 0
test_count = 1000
TEST = false # zmienić na true, żeby uruchomić testy

error = Array{BigFloat}(test_count)
max_error = BigFloat(0)
min_error = BigFloat(Inf)

if TEST for p = 1:test_count
    if p%(test_count/100)==0
        @printf "%d " (100*p/test_count)
    end
    n = 10
    temp_data, temp_solution₁ = gen_test(n)
    
    function temp_f(x::Array{BigFloat})
        return ((-temp_data .+ x) .^ 2)' * [1, 1, 1, -C*C]
    end
    
    temp_∂farr = Array{Function, 2}(n, 4)
    for i = 1:n
        for j = 1:3
            temp_∂farr[i, j] = function(x) return 2*x[j] - 2*temp_data[j, i] end
        end
        temp_∂farr[i, 4] = function(x) return 2*C*C*temp_data[4, i] - 2*C*C*x[4] end
    end
    
    function temp_∂f(x::Array{BigFloat})
        return map((f) -> f(x), temp_∂farr)
    end
        
    function temp_S(x::Array{BigFloat})
        return [sum(4 * temp_f(x) .* (x[i] - temp_data[i, :])) for i = 1:4] .* [1, 1, 1, -C*C]
    end

    temp_∂Sarr = Array{Function, 2}(4, 4)
    for i = 1:4
        for j = 1:4
            temp_∂Sarr[i, j] = (i == 4 || j == 4) ? 
                function(x) return - C * C * 8 * sum((x[j] - temp_data[j, :]) .* (x[i] - temp_data[i, :])) end :
                function(x) return 8 * sum((x[j] - temp_data[j, :]) .* (x[i] - temp_data[i, :])) end

        end
    end

    for i = 1:3
        temp_∂Sarr[i, i] = function(x) return 4 * sum(temp_f(x) + (2 * (temp_data[i, :] - x[i]) .* (temp_data[i, :] - x[i]))) end
    end
    
    function temp_∂S(x::Array{BigFloat})
        return map((f) -> f(x), temp_∂Sarr)
    end
        
    temp_result = NewtonMethod(temp_S, temp_∂S, 1e-70; max_iter=50, log=false)
    
    error[p] = norm(temp_solution₁[1:3] - temp_result[1:3])
    if error[p] > max_error max_error = error[p] end
    if error[p] < min_error min_error = error[p] end
    if(error[p] < 1e-70) solved_count += 1 end
end 
    @printf "\nsolved: %d / %d" solved_count test_count
    @printf "\nerror: max: %.4e avg: %.4e min: %.4e" max_error mean(error) min_error
end


solved: 0 / 1000

LoadError: [91mUndefRefError: access to undefined reference[39m

## Ważona metoda Newtona

In [8]:
function NewtonMethod(f, ∂f, ϵ; x=Array{BigFloat}([0, 0, 0, 0]), max_iter=1000, log=true)
    i = 0
    while norm(f(x)) > ϵ && i < max_iter
        δ = (∂f(x)' * ∂f(x)) \ ∂f(x)' * f(x)
        x = x - δ
        i += 1
        if log prnt(abs.(x - solution₁), abs.(f(x))) end
    end
    return x
end

newton_all_result = NewtonMethod(f, ∂f, 1e-70; log=true)

@show norm(f(newton_all_result))

x - sol: [1.5653e-77, 9.9316e-77, 1.7272e-77, 1.8602e+00]	f(x): [9.8420e-01, 9.8420e-01, 9.8420e-01, 9.8420e-01]
x - sol: [5.3976e-78, 5.6135e-77, 8.6362e-78, 6.4692e+00]	f(x): [1.5326e+01, 1.5326e+01, 1.5326e+01, 1.5326e+01]
x - sol: [1.5005e-76, 2.8802e-75, 2.6772e-76, 2.4341e+00]	f(x): [3.5968e+00, 3.5968e+00, 3.5968e+00, 3.5968e+00]
x - sol: [3.5624e-77, 5.6135e-77, 3.4545e-77, 6.4939e-01]	f(x): [7.0358e-01, 7.0358e-01, 7.0358e-01, 7.0358e-01]
x - sol: [2.1590e-78, 4.3181e-78, 8.6362e-78, 7.5926e-02]	f(x): [7.2644e-02, 7.2644e-02, 7.2644e-02, 7.2644e-02]
x - sol: [3.6434e-78, 1.2954e-77, 0.0000e+00, 1.3081e-03]	f(x): [1.2300e-03, 1.2300e-03, 1.2300e-03, 1.2300e-03]
x - sol: [9.1759e-78, 1.7272e-77, 0.0000e+00, 4.0184e-07]	f(x): [3.7773e-07, 3.7773e-07, 3.7773e-07, 3.7773e-07]
x - sol: [5.3976e-78, 1.2954e-77, 0.0000e+00, 3.7897e-14]	f(x): [3.5670e-14, 3.5670e-14, 3.5670e-14, 3.5670e-14]
x - sol: [2.4289e-78, 2.5909e-77, 0.0000e+00, 4.9617e-17]	f(x): [3.1809e-28, 3.1809e-28, 3.1809e

1.029118255513150129808937272749479811493807359761588150742437921715647832621081e-76

### Test ważonej metody Newtona

In [9]:
solved_count = 0
test_count = 100000
TEST = false # zmienić na true, żeby uruchomić testy

error = Array{BigFloat}(test_count)
max_error = BigFloat(0)
min_error = BigFloat(Inf)

if TEST for p = 1:test_count
    if p%(test_count/100)==0
        @printf "%d " (100*p/test_count)
    end
    n = 10
    temp_data, temp_solution₁ = gen_test(n)
    
    function temp_f(x::Array{BigFloat})
        return ((-temp_data .+ x) .^ 2)' * [1, 1, 1, -C*C]
    end
    
    temp_∂farr = Array{Function, 2}(n, 4)
    for i = 1:n
        for j = 1:3
            temp_∂farr[i, j] = function(x) return 2*x[j] - 2*temp_data[j, i] end
        end
        temp_∂farr[i, 4] = function(x) return 2*C*C*temp_data[4, i] - 2*C*C*x[4] end
    end
    
    function temp_∂f(x::Array{BigFloat})
        return map((f) -> f(x), temp_∂farr)
    end
    
    temp_result = NewtonMethod(temp_f, temp_∂f, 1e-70; log=false)
    
    error[p] = norm(temp_solution₁[1:3] - temp_result[1:3])
    if error[p] > max_error max_error = error[p] end
    if error[p] < min_error min_error = error[p] end
    if(error[p] < 1e-70) solved_count += 1 end
end
    @printf "\nsolved: %d / %d" solved_count test_count
    @printf "\nerror: max: %.4e avg: %.4e min: %.4e" max_error mean(error) min_error
end


solved: 0 / 100000

LoadError: [91mUndefRefError: access to undefined reference[39m